In [1]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments


2025-02-23 15:36:05.297066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740324965.319971 2596752 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740324965.327326 2596752 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 15:36:05.351838: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

# Load data to panda data frame
rows=[]
with open(os.path.join('dontpatronizeme_pcl.tsv')) as f:
    for line in f.readlines()[4:]:
        par_id=line.strip().split('\t')[0]
        art_id = line.strip().split('\t')[1]
        keyword=line.strip().split('\t')[2]
        country=line.strip().split('\t')[3]
        t=line.strip().split('\t')[4]#.lower()
        l=line.strip().split('\t')[-1]
        if l=='0' or l=='1':
            lbin=0
        else:
            lbin=1
        rows.append(
            {'par_id':par_id,
            'art_id':art_id,
            'keyword':keyword,
            'country':country,
            'text':t, 
            'label':lbin, 
            'orig_label':l
            }
            )
df=pd.DataFrame(rows, columns=['par_id', 'art_id', 'keyword', 'country', 'text', 'label', 'orig_label']) 
print(df.shape[0])
print(df[df['label'] == 0].shape[0])
print(df[df['label'] == 1].shape[0])


10469
9476
993


In [3]:

# Filter rows with a specific label (e.g., 'negative')
noPCL = df[df['label'] == 0]
PCL = df[df['label'] == 1]
average = df

# Get sentence length (in words)
noPCL['text_Length'] = noPCL['text'].apply(lambda x: len(x.split()))
PCL['text_Length'] = PCL['text'].apply(lambda x: len(x.split()))
average['text_length']  = df['text'].apply(lambda x: len(x.split()))

# Calculate average length
average_length_noPCL = noPCL['text_Length'].mean()
average_length_PCL = PCL['text_Length'].mean()
average_l = average['text_length'].mean()

print(average_length_noPCL)
print(average_length_PCL)
print(average_l)


47.87779653862389
53.620342396777446
48.42248543318369


/tmp/ipykernel_2596752/4254412181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noPCL['text_Length'] = noPCL['text'].apply(lambda x: len(x.split()))
/tmp/ipykernel_2596752/4254412181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PCL['text_Length'] = PCL['text'].apply(lambda x: len(x.split()))


In [4]:
# Group by 'label' and 'keyword', get counts
counts = df.groupby(['label', 'keyword']).size().reset_index(name='Count')

# Display the first few rows to confirm
print(counts.head())

# Get total rows per label
label_totals = df['label'].value_counts()

# Map the total count for each 'label' to the 'Count' DataFrame
counts['Label_Total'] = counts['label'].map(label_totals)

# Calculate percentage
counts['Percentage'] = (counts['Count'] / counts['Label_Total']) * 100

# Optional: Drop helper column if you don't need it
counts.drop(columns=['Label_Total'], inplace=True)

print(counts)


   label    keyword  Count
0      0   disabled    947
1      0   homeless    899
2      0   hopeless    881
3      0  immigrant   1031
4      0    in-need    906
    label        keyword  Count  Percentage
0       0       disabled    947    9.993668
1       0       homeless    899    9.487125
2       0       hopeless    881    9.297172
3       0      immigrant   1031   10.880118
4       0        in-need    906    9.560996
5       0        migrant   1053   11.112284
6       0  poor-families    759    8.009709
7       0        refugee    982   10.363022
8       0     vulnerable   1000   10.552976
9       0          women   1018   10.742930
10      1       disabled     81    8.157100
11      1       homeless    178   17.925478
12      1       hopeless    124   12.487412
13      1      immigrant     30    3.021148
14      1        in-need    176   17.724068
15      1        migrant     36    3.625378
16      1  poor-families    150   15.105740
17      1        refugee     86    8.660624
18

In [5]:
# Group by 'label' and 'keyword', get counts
counts = df.groupby(['label', 'country']).size().reset_index(name='Count')

# Display the first few rows to confirm
print(counts.head())

# Get total rows per label
label_totals = df['label'].value_counts()

# Map the total count for each 'label' to the 'Count' DataFrame
counts['Label_Total'] = counts['label'].map(label_totals)

# Calculate percentage
counts['Percentage'] = (counts['Count'] / counts['Label_Total']) * 100

# Optional: Drop helper column if you don't need it
counts.drop(columns=['Label_Total'], inplace=True)

print(counts)

   label country  Count
0      0      au    504
1      0      bd    468
2      0      ca    484
3      0      gb    483
4      0      gh    448
    label country  Count  Percentage
0       0      au    504    5.318700
1       0      bd    468    4.938793
2       0      ca    484    5.107640
3       0      gb    483    5.097087
4       0      gh    448    4.727733
5       0      hk    461    4.864922
6       0      ie    485    5.118193
7       0      in    491    5.181511
8       0      jm    433    4.569439
9       0      ke    494    5.213170
10      0      lk    455    4.801604
11      0      my    503    5.308147
12      0      ng    465    4.907134
13      0      nz    471    4.970452
14      0      ph    478    5.044322
15      0      pk    492    5.192064
16      0      sg    497    5.244829
17      0      tz    374    3.946813
18      0      us    499    5.265935
19      0      za    491    5.181511
20      1      au     37    3.726083
21      1      bd     44    4.431017
22   

In [6]:
# Split in to dev set and test set
train_ids = pd.read_csv('train_semeval_parids-labels.csv')
dev_ids = pd.read_csv('dev_semeval_parids-labels.csv')
train_ids = train_ids.iloc[:, 0].astype(str).tolist()
dev_ids = dev_ids.iloc[:, 0].astype(str).tolist()

train_df = df[df['par_id'].isin(train_ids)]
dev_df = df[df['par_id'].isin(dev_ids)]
print(len(train_df)+len(dev_df))


10469


In [7]:
KEYWORDS = df['keyword'].unique().tolist()
COUNTRIES = df['country'].unique().tolist()

In [8]:
'''
class PCLDataset(Dataset):
    def __init__(self, texts, labels, keywords, countries, tokenizer, max_length=128, shuffle=True):
        if shuffle:
            combined = list(zip(texts, labels, keywords, countries))
            random.shuffle(combined)
            texts, labels, keywords, countries = zip(*combined)
            texts, labels, keywords, countries = list(texts), list(labels), list(keywords), list(countries)
        
        # Tokenize texts
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels
        # Compute extra features for each text (e.g. sentence length and keyword indicator)
        #self.extra_features = [self.compute_extra_features(text) for text in texts]
        #self.extra_features = [self.extract_features(text, keyword, country) for text, keyword, country in zip(texts, keywords, countries)]
        
    def extract_features(self, text, keyword, country):
        length = len(text.split())

        # One-hot keyword presence
        keyword_presence = [1 if keyword == kw else 0 for kw in KEYWORDS]

        # One-hot encoding of country
        country_encoding = [1 if country == c else 0 for c in COUNTRIES]

        # Combine all features
        features = [length] + keyword_presence + country_encoding
        return features
    
    def __getitem__(self, idx):
        # Get tokenized text
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Get label
        item['labels'] = torch.tensor(self.labels[idx])
        # Get extra features (convert to float tensor)
        extra_feature = torch.tensor(self.extra_features[idx], dtype=torch.float)
        item['extra_features'] = extra_feature
        return item

    def __len__(self):
        return len(self.labels)
'''

"\nclass PCLDataset(Dataset):\n    def __init__(self, texts, labels, keywords, countries, tokenizer, max_length=128, shuffle=True):\n        if shuffle:\n            combined = list(zip(texts, labels, keywords, countries))\n            random.shuffle(combined)\n            texts, labels, keywords, countries = zip(*combined)\n            texts, labels, keywords, countries = list(texts), list(labels), list(keywords), list(countries)\n        \n        # Tokenize texts\n        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)\n        self.labels = labels\n        # Compute extra features for each text (e.g. sentence length and keyword indicator)\n        #self.extra_features = [self.compute_extra_features(text) for text in texts]\n        #self.extra_features = [self.extract_features(text, keyword, country) for text, keyword, country in zip(texts, keywords, countries)]\n        \n    def extract_features(self, text, keyword, country):\n        lengt

In [9]:
'''
class PCLModel(nn.Module):
    def __init__(self, model_name, num_labels, extra_feature_dim):
        super(PCLModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        hidden_size = self.model.config.hidden_size  # typically 768 for roberta-base
        self.dropout = nn.Dropout(0.1)
        # Classifier takes concatenated [CLS] embedding and extra features
        #self.classifier = nn.Linear(hidden_size + extra_feature_dim, num_labels)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size + extra_feature_dim, 256),  
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask, extra_features, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output  # [CLS] token representation
        # Concatenate the extra features
        combined = torch.cat((cls_output, extra_features), dim=1)
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits}
'''

'\nclass PCLModel(nn.Module):\n    def __init__(self, model_name, num_labels, extra_feature_dim):\n        super(PCLModel, self).__init__()\n        self.model = AutoModel.from_pretrained(model_name)\n        hidden_size = self.model.config.hidden_size  # typically 768 for roberta-base\n        self.dropout = nn.Dropout(0.1)\n        # Classifier takes concatenated [CLS] embedding and extra features\n        #self.classifier = nn.Linear(hidden_size + extra_feature_dim, num_labels)\n        self.classifier = nn.Sequential(\n            nn.Linear(hidden_size + extra_feature_dim, 256),  \n            nn.ReLU(),\n            nn.Dropout(0.1),\n            nn.Linear(256, num_labels)\n        )\n\n    def forward(self, input_ids, attention_mask, extra_features, labels=None):\n        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)\n        cls_output = outputs.pooler_output  # [CLS] token representation\n        # Concatenate the extra features\n        combined = tor

In [ ]:
class PCLDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128, shuffle=True):
        # Tokenize texts
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels
        # Compute extra features for each text (e.g. sentence length and keyword indicator)
        #self.extra_features = [self.compute_extra_features(text) for text in texts]
        #self.extra_features = [self.extract_features(text, keyword, country) for text, keyword, country in zip(texts, keywords, countries)]
        
    def extract_features(self, text, keyword, country):
        length = len(text.split())

        # One-hot keyword presence
        keyword_presence = [1 if keyword == kw else 0 for kw in KEYWORDS]

        # One-hot encoding of country
        country_encoding = [1 if country == c else 0 for c in COUNTRIES]

        # Combine all features
        features = [length] + keyword_presence + country_encoding
        return features
    
    def __getitem__(self, idx):
        # Get tokenized text
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Get label
        item['labels'] = torch.tensor(self.labels[idx])
        # Get extra features (convert to float tensor)
        #extra_feature = torch.tensor(self.extra_features[idx], dtype=torch.float)
        #item['extra_features'] = extra_feature
        return item

    def __len__(self):
        return len(self.labels)
    

class PCLModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super(PCLModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        hidden_size = self.model.config.hidden_size  # typically 768 for roberta-base
        self.dropout = nn.Dropout(0.1)
        # Classifier takes concatenated [CLS] embedding and extra features
        #self.classifier = nn.Linear(hidden_size + extra_feature_dim, num_labels)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),  
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        #cls_output = outputs.pooler_output  # [CLS] token representation
        cls_output = outputs.last_hidden_state[:, 0, :]  # Take the [CLS] token representation
        # Concatenate the extra features
        combined = cls_output
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits}

In [11]:
# Load pretrained tokenizer and model
#model_name = "roberta-base"  # You can switch to "roberta-large" if desired
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load the tokenizer (if not already loaded)
#model_name = "roberta-base" # cased
model_name = "microsoft/deberta-base"  # DeBERTa-base (cased)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the custom model
num_labels = 2  # binary classification
extra_feature_dim = 1 + len(KEYWORDS) + len(COUNTRIES)
#model = PCLModel(model_name, num_labels, extra_feature_dim)
model = PCLModel(model_name, num_labels)

In [12]:
#train_texts = train_df["text"].tolist()
#train_labels = train_df["label"].tolist()

# Split train set into train and valiadation (90/10)
#train_texts, val_texts, train_labels, val_labels = train_test_split(
#    train_texts, train_labels, test_size=0.1, random_state=42
#)

# Create dataset objects for training and validation
#train_dataset = PCLDataset(train_texts, train_labels, tokenizer, max_length=128)
#val_dataset = PCLDataset(val_texts, val_labels, tokenizer, max_length=128)

In [13]:
import nlpaug.augmenter.word as naw
from sklearn.model_selection import train_test_split

# Split the original data into training and validation sets *before* augmentation
train_texts, val_texts, train_labels, val_labels, train_keywords, val_keywords, train_countries, val_countries = train_test_split(
    train_df["text"].tolist(),
    train_df["label"].tolist(),
    train_df["keyword"].tolist(),
    train_df["country"].tolist(),
    test_size=0.1,  # 20% for validation
    random_state=42
)

# Separate majority and minority classes (assuming label 1 is minority)
minority_indices = [i for i, label in enumerate(train_labels) if label == 1]
minority_texts = [train_texts[i] for i in minority_indices]
minority_keywords = [train_keywords[i] for i in minority_indices]
minority_countries = [train_countries[i] for i in minority_indices]

# Data augmentation with Contextual BERT substitution
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased',
    action="substitute",  # or "insert"
    device='cuda'  # or 'cpu'
)

augmented_texts = []
augmented_keywords = []
augmented_countries = []
num_aug = 9
for text, keyword, country in zip(minority_texts, minority_keywords, minority_countries):
    # Generate 9 augmentations per minority sample
    augmented_texts.extend([aug.augment(text) for _ in range(num_aug)])
    augmented_keywords.extend([keyword] * num_aug)
    augmented_countries.extend([country] * num_aug)

augmented_labels = [1] * len(augmented_texts)

# Combine augmented minority samples with the original training data (not validation)
train_texts += augmented_texts
train_labels += augmented_labels
train_keywords += augmented_keywords
train_countries += augmented_countries

# Ensure train_texts is a flat list (if augmentation returns nested lists)
train_texts = [text if isinstance(text, str) else text[0] for text in train_texts]

# Create dataset objects
#train_dataset = PCLDataset(train_texts, train_labels, train_keywords, train_countries, tokenizer, max_length=128)
#val_dataset = PCLDataset(val_texts, val_labels, val_keywords, val_countries, tokenizer, max_length=128)
train_dataset = PCLDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = PCLDataset(val_texts, val_labels, tokenizer, max_length=128)

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

Train samples: 13891, Validation samples: 838


In [14]:
print("Class distribution after augmentation:")
print(f"Class 0: {train_labels.count(0)} instances")
print(f"Class 1: {train_labels.count(1)} instances")

Class distribution after augmentation:
Class 0: 6831 instances
Class 1: 7060 instances


In [15]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',            # Output directory for model checkpoints
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=16,    # Batch size per device during training
    per_device_eval_batch_size=16,     # Batch size for evaluation
    learning_rate=2e-5,                # Initial learning rate
    warmup_steps=500,                  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # Strength of weight decay
    logging_dir='./logs',              # Directory for storing logs
    logging_steps=10,                  # Log every 10 steps
    evaluation_strategy="steps",       # Evaluate every 'eval_steps'
    eval_steps=100,                    # Evaluate every 100 steps
    save_steps=100,                    # Save checkpoint every 100 steps
    load_best_model_at_end=True,       # Load the best model at the end
    metric_for_best_model="eval_loss",
    greater_is_better=False,           # Lower eval_loss is better
    disable_tqdm=False,
    report_to=[]
)


/vol/bitbucket/eww24/nlpenv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Initialize and start the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement for 3 evaluations
)

trainer.train()

Step,Training Loss,Validation Loss
100,0.484700,0.769597
200,0.185500,0.391098
300,0.269800,0.311611
400,0.245800,0.325564
500,0.172100,0.381393
600,0.081700,0.260660
700,0.127500,0.298632
800,0.110100,0.232367
900,0.185900,0.229247
1000,0.107800,0.247956


TrainOutput(global_step=1800, training_loss=0.1751545145859321, metrics={'train_runtime': 851.0095, 'train_samples_per_second': 48.969, 'train_steps_per_second': 3.063, 'total_flos': 0.0, 'train_loss': 0.1751545145859321, 'epoch': 2.0713463751438437})

In [17]:
# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.21604157984256744, 'eval_runtime': 5.6897, 'eval_samples_per_second': 147.285, 'eval_steps_per_second': 9.315, 'epoch': 2.0713463751438437}


In [18]:
# Save the fine-tuned model and tokenizer
save_directory = "./fine_tuned_pcl_model"
trainer.save_model(save_directory)  # This saves the model to the specified directory
tokenizer.save_pretrained(save_directory)
print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to ./fine_tuned_pcl_model


In [19]:
# Load the saved model and tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_directory)

#loaded_tokenizer = AutoTokenizer.from_pretrained("./best_model")
#loaded_model = AutoModelForSequenceClassification.from_pretrained("./best_model")

Some weights of the model checkpoint at ./fine_tuned_pcl_model were not used when initializing RobertaForSequenceClassification: ['classifier.0.bias', 'classifier.0.weight', 'classifier.3.bias', 'classifier.3.weight', 'model.embeddings.LayerNorm.bias', 'model.embeddings.LayerNorm.weight', 'model.embeddings.word_embeddings.weight', 'model.encoder.layer.0.attention.output.LayerNorm.bias', 'model.encoder.layer.0.attention.output.LayerNorm.weight', 'model.encoder.layer.0.attention.output.dense.bias', 'model.encoder.layer.0.attention.output.dense.weight', 'model.encoder.layer.0.attention.self.in_proj.weight', 'model.encoder.layer.0.attention.self.pos_proj.weight', 'model.encoder.layer.0.attention.self.pos_q_proj.bias', 'model.encoder.layer.0.attention.self.pos_q_proj.weight', 'model.encoder.layer.0.attention.self.q_bias', 'model.encoder.layer.0.attention.self.v_bias', 'model.encoder.layer.0.intermediate.dense.bias', 'model.encoder.layer.0.intermediate.dense.weight', 'model.encoder.layer.0.o

In [20]:
dev_texts = dev_df["text"].tolist()
dev_labels = dev_df["label"].tolist()
dev_keywords = dev_df["keyword"].tolist()
dev_countries = dev_df["country"].tolist()


#dev_dataset = PCLDataset(dev_texts, dev_labels, dev_keywords, dev_countries, tokenizer, max_length=128)
dev_dataset = PCLDataset(dev_texts, dev_labels, tokenizer, max_length=128)


In [21]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Get predictions using the trainer
pred_output = trainer.predict(dev_dataset)
pred_labels = np.argmax(pred_output.predictions, axis=1)

accuracy = accuracy_score(dev_labels, pred_labels)
f1 = f1_score(dev_labels, pred_labels, average='binary')
precision = precision_score(dev_labels, pred_labels, average='binary')
recall = recall_score(dev_labels, pred_labels, average='binary')

print("Dev Accuracy:", accuracy)
print("Dev F1:", f1)
print("Dev Precision:", precision)
print("Dev Recall:", recall)

# Write predicted class (0 or 1) to dev.txt, one prediction per line
with open("dev.txt", "w") as f:
    for pred in pred_labels:
        f.write(f"{pred}\n")

Dev Accuracy: 0.9202483285577842
Dev F1: 0.5322128851540616
Dev Precision: 0.6012658227848101
Dev Recall: 0.47738693467336685


In [22]:
# Load the final test set
rows=[]
with open(os.path.join('task4_test.tsv')) as f:
    for line in f.readlines():
        par_id=line.strip().split('\t')[0]
        art_id = line.strip().split('\t')[1]
        keyword=line.strip().split('\t')[2]
        country=line.strip().split('\t')[3]
        t=line.strip().split('\t')[4]#.lower()
        rows.append(
            {'par_id':par_id,
            'art_id':art_id,
            'keyword':keyword,
            'country':country,
            'text':t, 
            }
            )
test_df=pd.DataFrame(rows, columns=['par_id', 'art_id', 'keyword', 'country', 'text']) 
print(test_df.shape[0])

test_texts = test_df["text"].tolist()
test_labels = [0] * len(test_texts)
test_keywords = test_df["keyword"].tolist()
test_countries = test_df["country"].tolist()

#test_dataset = PCLDataset(test_texts, test_labels, test_keywords, test_countries, tokenizer, max_length=128)
test_dataset = PCLDataset(test_texts, test_labels, tokenizer, max_length=128)


3832


In [23]:
pred_output = trainer.predict(test_dataset)
pred_labels = np.argmax(pred_output.predictions, axis=1)

print(len(pred_labels))

# Write predicted class (0 or 1) to test.txt, one prediction per line
with open("test.txt", "w") as f:
    for pred in pred_labels:
        f.write(f"{pred}\n")

3832


In [24]:
'''
# Use the loaded model to make predictions
from sklearn.metrics import f1_score

loaded_model.eval()
correct = 0
predictions = []
for i in range(len(dev_texts)):
    text = dev_texts[i]
    encodings = loaded_tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")
    loaded_model.eval()
    with torch.no_grad():
        outputs = loaded_model(**encodings)
        pred_label = torch.argmax(outputs.logits, dim=1)
        if pred_label.tolist()[0] == dev_labels[i]:
            correct += 1
        #print("Predictions:", predictions.tolist()[0], "Actual label:", dev_labels[i])
        predictions.append(pred_label.tolist()[0])
print(correct)
accuracy = correct/len(dev_texts)
f1 = f1_score(dev_labels, predictions, average='binary')
print("Accuracy:", accuracy, "F1:", f1)
'''

'\n# Use the loaded model to make predictions\nfrom sklearn.metrics import f1_score\n\nloaded_model.eval()\ncorrect = 0\npredictions = []\nfor i in range(len(dev_texts)):\n    text = dev_texts[i]\n    encodings = loaded_tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")\n    loaded_model.eval()\n    with torch.no_grad():\n        outputs = loaded_model(**encodings)\n        pred_label = torch.argmax(outputs.logits, dim=1)\n        if pred_label.tolist()[0] == dev_labels[i]:\n            correct += 1\n        #print("Predictions:", predictions.tolist()[0], "Actual label:", dev_labels[i])\n        predictions.append(pred_label.tolist()[0])\nprint(correct)\naccuracy = correct/len(dev_texts)\nf1 = f1_score(dev_labels, predictions, average=\'binary\')\nprint("Accuracy:", accuracy, "F1:", f1)\n'

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(
    train_df['text'], train_df['label'], test_size=0.3, random_state=42, stratify=train_df['label']
)

# Initialize the CountVectorizer (Bag-of-Words)
vectorizer = CountVectorizer(stop_words='english')
X_train_bow = vectorizer.fit_transform(train_df['text'])
X_test_bow = vectorizer.transform(dev_df['text'])

# Initialize and train a simple classifier. Here we use Logistic Regression.
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train_bow, train_df['label'])

# Make predictions on the test set
y_pred = clf.predict(X_test_bow)

# Evaluate the model
print("Accuracy:", accuracy_score(dev_df['label'], y_pred))
print("\nClassification Report:\n", classification_report(dev_df['label'], y_pred))

# Optionally: inspect the top features associated with PCL detection.
feature_names = np.array(vectorizer.get_feature_names_out())
coef = clf.coef_[0]
# Get the indices of the top 10 features for PCL (i.e., with highest positive coefficients)
top_positive_indices = coef.argsort()[-10:][::-1]
print("\nTop features for PCL detection:")
for idx in top_positive_indices:
    print(f"{feature_names[idx]}: {coef[idx]:.4f}")

Accuracy: 0.897803247373448

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.95      1895
           1       0.41      0.18      0.25       199

    accuracy                           0.90      2094
   macro avg       0.67      0.58      0.60      2094
weighted avg       0.87      0.90      0.88      2094


Top features for PCL detection:
christmas: 2.1195
dreamers: 1.8463
hungry: 1.7233
hope: 1.6972
dignity: 1.5203
donate: 1.4944
duty: 1.4201
delivered: 1.4135
underprivileged: 1.4076
lives: 1.3839


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.3, random_state=42, stratify=df['label']
)

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text'])
X_test_tfidf = tfidf_vectorizer.transform(dev_df['text'])

# Initialize and train a logistic regression classifier.
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train_tfidf, train_df['label'])

# Make predictions on the test set
y_pred = clf.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(dev_df['label'], y_pred))
print("\nClassification Report:\n", classification_report(dev_df['label'], y_pred))

# Optionally: inspect the top features associated with PCL detection.
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
coef = clf.coef_[0]
# Get the indices of the top 10 features for PCL (i.e., with highest positive coefficients)
top_positive_indices = coef.argsort()[-10:][::-1]
print("\nTop features for PCL detection:")
for idx in top_positive_indices:
    print(f"{feature_names[idx]}: {coef[idx]:.4f}")

Accuracy: 0.9059216809933143

Classification Report:
               precision    recall  f1-score   support

           0       0.91      1.00      0.95      1895
           1       0.60      0.03      0.06       199

    accuracy                           0.91      2094
   macro avg       0.75      0.51      0.50      2094
weighted avg       0.88      0.91      0.87      2094


Top features for PCL detection:
hope: 3.6782
lives: 3.1356
help: 3.1244
homeless: 3.1015
need: 3.0978
life: 3.0676
poor: 2.8790
christmas: 2.8656
children: 2.5291
poverty: 2.4010
